In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
import pandas as pd
from utils import (
    df_to_excel_col_dict,
    filter_tnc_only,
    getSharesIdx,
    getSharesIdxCI95,
    link_dt,
    load_hh_raw,
    load_person_reformat,
    load_trip_assign_day,
    load_trip_raw,
    out_dir,
    prep_data_1d,
    prep_data_2d,
    reformat_dir,
    survey_processed_dir,
    tour_extract_allwk_dir,
    tour_extract_wkday_dir,
    trip_join_hh_person,
    write_to_excel,
)
from xlsxwriter.utility import xl_rowcol_to_cell


In [ ]:
hh_raw_filepath = survey_processed_dir / "hh.csv"
person_reformat_filepath = reformat_dir / "person-reformat.csv"
out_filepath = out_dir / "03_TNCTrips_Purpose.xlsx"

In [ ]:
weighted = False

In [ ]:
hh = load_hh_raw(hh_raw_filepath)
person = load_person_reformat(person_reformat_filepath)
trip_raw = trip_join_hh_person(
    load_trip_raw(tnc=True, transit_access_egress=False), hh, person
)
# also load weight vars, but weights not available yet:
# "wt_alladult_mon",
# "wt_alladult_tue",
# "wt_alladult_wed",
# "wt_alladult_thu",
# "wt_alladult_fri",
# "wt_alladult_sat",
# "wt_alladult_sun",

In [ ]:
def process_trip(trip_assign_day_filepath, trip_raw):
    trip = filter_tnc_only(
        load_trip_assign_day(trip_assign_day_filepath),
        trip_raw,
    ).to_pandas()
    trip = link_dt(trip)
    return trip

In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
format1 = workbook.add_format({"num_format": "#,##0.0"})

In [ ]:
# process weekdat trip records
trip = process_trip(tour_extract_wkday_dir / "trip-assign_day.csv", trip_raw)

In [ ]:
row = 0
sname = "Weekday"
d1_dict = df_to_excel_col_dict["dpurp"]

title = "TNC Trips by " + d1_dict["desc"]
tab = prep_data_1d(
    trip,
    d1_dict["desc"],
    d1_dict["col"],
    "trexpfac",
    d1_dict["vals"],
    d1_dict["labels"],
)
row = write_to_excel(writer, tab, sname, title, row)

title = "Column Shares by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

tab2 = tab.copy()
tab2.iloc[-1, -1] = tab2.iloc[-1, 0]

title = "Column Shares 95% CI by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdxCI95(tab.copy()), sname, title, row)

tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
wt_cols = ["count", "trexpfac"]
wt_desc = ["(Unweighted)", "(Weighted)"]

# TODO add back raceeth
for key in ["depart_hour", "hh_income_broad_cat", "pagey", "pgend", "tnc_type"]:
    d2_dict = df_to_excel_col_dict[key]
    row = 0
    sname = d2_dict["desc"]

    for wc, wd in zip(wt_cols, wt_desc):
        title = "TNC Trips by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        tab = prep_data_2d(
            trip,
            d1_dict["col"],
            d1_dict["vals"],
            d1_dict["labels"],
            d2_dict["col"],
            d2_dict["vals"],
            d2_dict["labels"],
            wc,
        )
        row = write_to_excel(writer, tab.astype("float64"), sname, title, row)

        if wc == "count":
            tab2 = tab.copy()
        else:
            tab2.iloc[:-1, :-1] = tab.iloc[:-1, :-1]

        title = (
            "Column Shares by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        )
        row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

        title = (
            "Column Shares 95% CI by "
            + d2_dict["desc"]
            + " and "
            + d1_dict["desc"]
            + " "
            + wd
        )
        row = write_to_excel(writer, getSharesIdxCI95(tab2.copy()), sname, title, row)
    tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
## Process all week trip records
trip = process_trip(tour_extract_allwk_dir / "trip-assign_day.csv", trip_raw)

if weighted:
    DOW_LOOKUP = {1: "mon", 2: "tue", 3: "wed", 4: "thu", 5: "fri", 6: "sat", 7: "sun"}
    trip["trexpfac"] = 0
    for dow_num, dow in DOW_LOOKUP.items():
        trip.loc[trip["day"] == dow_num, "trexpfac"] = trip.loc[
            trip["day"] == dow_num, "wt_alladult_" + dow
        ]
    trip["trexpfac"] = trip["trexpfac"].fillna(0)

In [ ]:
row = 0
sname = "DOW"
d2_dict = df_to_excel_col_dict["day"]

In [ ]:
for wc, wd in zip(wt_cols, wt_desc):
    title = "TNC Trips by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
    tab = prep_data_2d(
        trip,
        d1_dict["col"],
        d1_dict["vals"],
        d1_dict["labels"],
        d2_dict["col"],
        d2_dict["vals"],
        d2_dict["labels"],
        wc,
    )
    row = write_to_excel(writer, tab.astype("float64"), sname, title, row)

    if wc == "count":
        tab2 = tab.copy()
    else:
        tab2.iloc[:-1, :-1] = tab.iloc[:-1, :-1]

    title = "Column Shares by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
    row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

    title = (
        "Column Shares 95% CI by "
        + d2_dict["desc"]
        + " and "
        + d1_dict["desc"]
        + " "
        + wd
    )
    row = write_to_excel(writer, getSharesIdxCI95(tab2.copy()), sname, title, row)
tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
writer.close()